<a href="https://colab.research.google.com/github/missestrella2/scriptscolab/blob/main/extrae_mejora_audio_y_transcribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El _notebook_:

1. **Encuentra** el último video en la carpeta
2. **Extrae** el audio en fragmentos y los almacena  
3. **Procesa** cada fragmento y lo mejora
4. **Reemplaza** la pista original del video, generando UN VIDEO NUEVO CON AUDIO MEJORADO
5. **Transcribe** el video (luego hay que procesarlo para corregirlo, idealmente con chatgpt y por ultimo a mano) generando un txt

In [ ]:
# prompt: montar drive
from google.colab import drive

drive.flush_and_unmount()  # Desmonta Google Drive

drive.mount('/content/drive')


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
import subprocess
import os



# Lista de librerías a instalar
librerias = ["librosa", "numpy", "soundfile", "whisper", "torch", "pydub", "pyloudnorm","openai-whisper"]

# Función para instalar y mostrar mensajes de inicio y finalización
def instalar_libreria(libreria):
    print(f"📦 Instalando {libreria}...")
    subprocess.run(["pip", "install", libreria], stdout=open(os.devnull, 'w'), stderr=open(os.devnull, 'w'))
    print(f"✅ {libreria} instalada correctamente.")

# Instalar todas las librerías
for lib in librerias:
    instalar_libreria(lib)

print("🎉 Todas las librerías han sido instaladas correctamente.")


📦 Instalando librosa...
✅ librosa instalada correctamente.
📦 Instalando numpy...
✅ numpy instalada correctamente.
📦 Instalando soundfile...
✅ soundfile instalada correctamente.
📦 Instalando whisper...
✅ whisper instalada correctamente.
📦 Instalando torch...
✅ torch instalada correctamente.
📦 Instalando pydub...
✅ pydub instalada correctamente.
📦 Instalando pyloudnorm...
✅ pyloudnorm instalada correctamente.
📦 Instalando openai-whisper...
✅ openai-whisper instalada correctamente.
🎉 Todas las librerías han sido instaladas correctamente.


In [ ]:
#!pip install librosa numpy soundfile whisper torch pydub pyloudnorm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 17.1 MB/s eta 0:00:00
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sh

In [ ]:
import os
import librosa
import numpy as np
import soundfile as sf
import whisper
import torch
from pydub import AudioSegment
from pydub import effects
import pyloudnorm as pyln  # Para normalización LUFS


In [ ]:
# Definir carpeta raíz para organizar los archivos
root_folder = "/content/drive/MyDrive/erick_videos"
os.makedirs(root_folder, exist_ok=True)

# Buscar el archivo MP4 más reciente en la carpeta raíz
def get_latest_video(folder):
    video_files = [f for f in os.listdir(folder) if f.endswith(".mp4")]
    if not video_files:
        raise FileNotFoundError("❌ No se encontraron archivos MP4 en la carpeta.")
    latest_video = max(video_files, key=lambda f: os.path.getmtime(os.path.join(folder, f)))
    return os.path.join(folder, latest_video)

# Definir rutas dinámicas basadas en el último video encontrado
video_path = get_latest_video(root_folder)
video_name = os.path.splitext(os.path.basename(video_path))[0]  # Obtener nombre del archivo sin extensión
video_folder = os.path.join(root_folder, video_name)

# Crear subcarpetas dentro del directorio del video
audio_sin_mejorar_folder = os.path.join(video_folder, "audios_sin_mejorar")
audio_mejorado_folder = os.path.join(video_folder, "audios_mejorados")
transcription_folder = os.path.join(video_folder, "transcripciones")
video_output_folder = os.path.join(video_folder, "videos_con_audio_mejorado")
processed_audio_path = os.path.join(audio_mejorado_folder, "audio_mejorado.wav")
processed_audio_aac = processed_audio_path.replace(".wav", ".aac")

# Crear carpetas si no existen
os.makedirs(audio_sin_mejorar_folder, exist_ok=True)
os.makedirs(audio_mejorado_folder, exist_ok=True)
os.makedirs(transcription_folder, exist_ok=True)
os.makedirs(video_output_folder, exist_ok=True)

# Definir rutas de archivos
audio_path = os.path.join(video_folder, "audio_extraido.wav")
processed_audio_path = os.path.join(video_folder, "audio_mejorado.wav")
transcription_path = os.path.join(transcription_folder, "transcripcion.txt")
final_video_path = os.path.join(video_output_folder, "video_con_audio_mejorado.mp4")
fragment_duration = 10 * 60 * 1000  # 10 minutos en milisegundos

# Imprimir rutas para verificación
print("📂 Rutas utilizadas:")
print(f"   🎥 Video original: {video_path}")
print(f"   📁 Carpeta del video: {video_folder}")
print(f"   🔉 Carpeta de audios sin mejorar: {audio_sin_mejorar_folder}")
print(f"   🔊 Carpeta de audios mejorados: {audio_mejorado_folder}")
print(f"   📝 Carpeta de transcripciones: {transcription_folder}")
print(f"   🎬 Carpeta de videos finales: {video_output_folder}")

📂 Rutas utilizadas:
   🎥 Video original: /content/drive/MyDrive/erick_videos/0823.mp4
   📁 Carpeta del video: /content/drive/MyDrive/erick_videos/0823
   🔉 Carpeta de audios sin mejorar: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar
   🔊 Carpeta de audios mejorados: /content/drive/MyDrive/erick_videos/0823/audios_mejorados
   📝 Carpeta de transcripciones: /content/drive/MyDrive/erick_videos/0823/transcripciones
   🎬 Carpeta de videos finales: /content/drive/MyDrive/erick_videos/0823/videos_con_audio_mejorado


In [ ]:
# Extraer audio del video
print(f"🎬 Extrayendo audio del video más reciente: {video_path}")
os.system(f"ffmpeg -i \"{video_path}\" -q:a 0 -map a \"{audio_path}\"")
if not os.path.exists(audio_path):
    raise FileNotFoundError("❌ Error: No se pudo extraer el audio del video. Verifica la ruta del video.")
print("✅ Audio extraído correctamente.")

🎬 Extrayendo audio del video más reciente: /content/drive/MyDrive/erick_videos/0823.mp4
✅ Audio extraído correctamente.


In [ ]:
# Cargar audio y dividirlo en fragmentos
print("🎧 Cargando y dividiendo el audio en fragmentos...")
audio = AudioSegment.from_wav(audio_path)
duracion = len(audio)  # Duración total en milisegundos
fragmentos = []

for i, start in enumerate(range(0, duracion, fragment_duration)):
    segmento = audio[start:start+fragment_duration]
    parte_path = os.path.join(audio_sin_mejorar_folder, f"parte_{i}.wav")
    segmento.export(parte_path, format="wav")
    fragmentos.append(parte_path)
    print(f"🟢 Fragmento guardado: {parte_path}")

print("✅ Audio dividido correctamente.")

🎧 Cargando y dividiendo el audio en fragmentos...
🟢 Fragmento guardado: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_0.wav
🟢 Fragmento guardado: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_1.wav
🟢 Fragmento guardado: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_2.wav
🟢 Fragmento guardado: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_3.wav
🟢 Fragmento guardado: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_4.wav
🟢 Fragmento guardado: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_5.wav
🟢 Fragmento guardado: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_6.wav
🟢 Fragmento guardado: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_7.wav
✅ Audio dividido correctamente.


In [ ]:
import os
import glob
import numpy as np
import pyloudnorm as pyln
from pydub import AudioSegment, effects

# Carpetas de entrada/salida (ajusta a tus rutas)
INPUT_FOLDER = os.path.join(video_folder, "audios_sin_mejorar")
OUTPUT_FOLDER = os.path.join(video_folder, "audios_mejorados")

def mejorar_audio(file_path, output_file):
    print(f"🔧 Procesando: {file_path}")

    # 1. Carga el audio con pydub (asumiendo .wav)
    audio = AudioSegment.from_wav(file_path)

    # 2. Filtro pasa-altos ~ 100 Hz para reducir ruidos graves
    audio = audio.high_pass_filter(100)

    # 3. Aplicar un filtro pasa-bajos para suavizar los agudos (bajo 10kHz)
    audio = audio.low_pass_filter(10000)

    # 4. **Aplicar compresión moderada como limitador**
    compressed_audio = effects.compress_dynamic_range(
        audio,
        threshold=-6.0,   # Antes: -60.0 dB → Ahora -6.0 dB (evita saturación)
        ratio=4.0,        # Antes: 15:1 → Ahora 4:1 (más natural)
        attack=5.0,       # Antes: 1.0 → Ahora 5 ms (más gradual)
        release=250.0     # Antes: 400.0 → Ahora 250 ms
    )

    # 5. **Normalizar volumen después de la compresión**
    normalized_audio = effects.normalize(compressed_audio)

    # 6. Convertir audio a numpy para análisis de loudness
    sample_width = normalized_audio.sample_width
    channels = normalized_audio.channels
    frame_rate = normalized_audio.frame_rate

    # Extraer muestras como array int
    samples = np.array(normalized_audio.get_array_of_samples())

    # Si hay más de un canal, convertir a mono para medición LUFS
    if channels > 1:
        samples = samples.reshape((-1, channels))
        samples = samples.mean(axis=1)

    # Convertir int -> float en el rango [-1.0, 1.0]
    max_sample_value = float(2 ** (8 * sample_width - 1))
    samples = samples / max_sample_value

    # 7. Medir Loudness (LUFS) con pyloudnorm
    meter = pyln.Meter(frame_rate)
    loudness_before = meter.integrated_loudness(samples)
    print(f"   📊 LUFS antes de normalizar: {loudness_before:.2f} LUFS")

    # 8. Normalizar a -14 LUFS con limitador de saturación
    target_lufs = -14.0
    samples_normalized = pyln.normalize.loudness(samples, loudness_before, target_lufs)

    # Aplicar un limitador suave para evitar clipping
    samples_normalized = np.clip(samples_normalized, -0.95, 0.95)  # Antes: [-1.0, 1.0]
    print(f"   ✅ Normalizado a {target_lufs} LUFS")

    # 9. Convertir de float a int16 para volver a pydub
    y_int16 = np.int16(samples_normalized * max_sample_value)

    # Construir un AudioSegment (MONO en este caso)
    final_audio = AudioSegment(
        y_int16.tobytes(),
        frame_rate=frame_rate,
        sample_width=sample_width,
        channels=1
    )

    # 10. Exportar a WAV
    final_audio.export(output_file, format="wav")
    print(f"   ✅ Archivo exportado en: {output_file}\n")


def procesar_carpeta():
    # Crear carpeta de salida si no existe
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    # Buscar todos los .wav en la carpeta de entrada
    archivos = glob.glob(os.path.join(INPUT_FOLDER, "*.wav"))
    if not archivos:
        print(f"No se encontraron archivos WAV en '{INPUT_FOLDER}'.")
        return

    print(f"🎛 Encontrados {len(archivos)} archivo(s) en '{INPUT_FOLDER}'. Iniciando proceso...\n")
    for i, archivo in enumerate(archivos, 1):
        # Nombre de salida
        nombre_salida = os.path.basename(archivo).replace(".wav", "_mejorado.wav")
        ruta_salida = os.path.join(OUTPUT_FOLDER, nombre_salida)

        # Mejorar
        mejorar_audio(archivo, ruta_salida)
        print(f"✅ Progreso: {i}/{len(archivos)} finalizado.\n")

    print("🏁 ¡Proceso completado!")


if __name__ == "__main__":
    procesar_carpeta()


🎛 Encontrados 8 archivo(s) en '/content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar'. Iniciando proceso...

🔧 Procesando: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_0.wav
   📊 LUFS antes de normalizar: -43.31 LUFS


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


   ✅ Normalizado a -14.0 LUFS
   ✅ Archivo exportado en: /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_0_mejorado.wav

✅ Progreso: 1/8 finalizado.

🔧 Procesando: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_1.wav
   📊 LUFS antes de normalizar: -19.26 LUFS


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


   ✅ Normalizado a -14.0 LUFS
   ✅ Archivo exportado en: /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_1_mejorado.wav

✅ Progreso: 2/8 finalizado.

🔧 Procesando: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_2.wav
   📊 LUFS antes de normalizar: -19.64 LUFS


...

/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


   ✅ Normalizado a -14.0 LUFS
   ✅ Archivo exportado en: /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_5_mejorado.wav

✅ Progreso: 6/8 finalizado.

🔧 Procesando: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_6.wav
   📊 LUFS antes de normalizar: -19.55 LUFS


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


   ✅ Normalizado a -14.0 LUFS
   ✅ Archivo exportado en: /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_6_mejorado.wav

✅ Progreso: 7/8 finalizado.

🔧 Procesando: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_7.wav
   📊 LUFS antes de normalizar: -17.58 LUFS
   ✅ Normalizado a -14.0 LUFS
   ✅ Archivo exportado en: /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_7_mejorado.wav

✅ Progreso: 8/8 finalizado.

🏁 ¡Proceso completado!


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


In [ ]:
#version 1
import os
import glob
import numpy as np
import pyloudnorm as pyln

from pydub import AudioSegment, effects

# Carpetas de entrada/salida (ajusta a tus rutas)
INPUT_FOLDER = os.path.join(video_folder, "audios_sin_mejorar")
OUTPUT_FOLDER = os.path.join(video_folder, "audios_mejorados")

def mejorar_audio(file_path, output_file):
    print(f"🔧 Procesando: {file_path}")

    # 1. Carga el audio con pydub (asumiendo .wav)
    audio = AudioSegment.from_wav(file_path)

    # 2. Filtro pasa-altos ~ 100 Hz (similar a usar "High Pass Filter" en Audacity)
    #    Esto ayuda a quitar ruidos graves o zumbidos.
    audio = audio.high_pass_filter(100)

    # 3. Compresión:
    #    Aproximación a lo que se ve en las capturas de Audacity:
    #      - Umbral (threshold) muy bajo (-60 dB),
    #      - Relación alta (15:1),
    #      - Ataque muy rápido (1 ms),
    #      - Decaimiento (release) largo (400 ms),
    #      - "Knee" no se configura directamente en pydub (lo maneja internamente).
    #
    #    pydub.compress_dynamic_range() expone estos parámetros:
    #        threshold, ratio, attack, release, makeup_gain, ...
    #    *No* es exactamente igual que Audacity, pero se acerca.
    compressed_audio = effects.compress_dynamic_range(
        audio,
        threshold=-60.0,    # Umbral (dB)
        ratio=15.0,         # Proporción (15:1)
        attack=1.0,         # ms (ataque rápido)
        release=400.0,      # ms (release largo)
        #makeup_gain=False
    )

    # 4. Convierto el audio comprimido a un array numpy para medir LUFS con pyloudnorm
    sample_width = compressed_audio.sample_width
    channels = compressed_audio.channels
    frame_rate = compressed_audio.frame_rate

    # Extraer muestras como array int
    samples = np.array(compressed_audio.get_array_of_samples())

    # Si hay más de un canal, lo promediamos a mono para medir loudness
    if channels > 1:
        samples = samples.reshape((-1, channels))
        samples = samples.mean(axis=1)

    # Convertir int -> float en el rango [-1.0, 1.0]
    max_sample_value = float(2 ** (8 * sample_width - 1))
    samples = samples / max_sample_value

    # 5. Medir Loudness (LUFS) con pyloudnorm
    meter = pyln.Meter(frame_rate)
    loudness_before = meter.integrated_loudness(samples)
    print(f"   📊 LUFS antes de normalizar: {loudness_before:.2f} LUFS")

    # 6. Normalizar a -14 LUFS (similar a “Normalizar (vía LUFS) a -14.0” en Audacity)
    target_lufs = -14.0
    samples_normalized = pyln.normalize.loudness(samples, loudness_before, target_lufs)
    print(f"   ✅ Normalizado a {target_lufs} LUFS")

    # 7. Convertir de float a int16 para volver a pydub
    y_int16 = np.int16(samples_normalized * max_sample_value)

    # Construir un AudioSegment (MONO en este caso)
    # Si necesitas estéreo, habría que duplicar canales, etc.
    final_audio = AudioSegment(
        y_int16.tobytes(),
        frame_rate=frame_rate,
        sample_width=sample_width,
        channels=1
    )

    # 8. Exportar a WAV
    final_audio.export(output_file, format="wav")
    print(f"   ✅ Archivo exportado en: {output_file}\n")


def procesar_carpeta():
    # Crear carpeta de salida si no existe
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    # Buscar todos los .wav en la carpeta de entrada
    archivos = glob.glob(os.path.join(INPUT_FOLDER, "*.wav"))
    if not archivos:
        print(f"No se encontraron archivos WAV en '{INPUT_FOLDER}'.")
        return

    print(f"🎛 Encontrados {len(archivos)} archivo(s) en '{INPUT_FOLDER}'. Iniciando proceso...\n")
    for i, archivo in enumerate(archivos, 1):
        # Nombre de salida
        nombre_salida = os.path.basename(archivo).replace(".wav", "_mejorado.wav")
        ruta_salida = os.path.join(OUTPUT_FOLDER, nombre_salida)

        # Mejorar
        mejorar_audio(archivo, ruta_salida)
        print(f"✅ Progreso: {i}/{len(archivos)} finalizado.\n")

    print("🏁 ¡Proceso completado!")


if __name__ == "__main__":
    procesar_carpeta()


🎛 Encontrados 8 archivo(s) en '/content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar'. Iniciando proceso...

🔧 Procesando: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_0.wav
   📊 LUFS antes de normalizar: -68.17 LUFS


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


   ✅ Normalizado a -14.0 LUFS
   ✅ Archivo exportado en: /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_0_mejorado.wav

✅ Progreso: 1/8 finalizado.

🔧 Procesando: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_1.wav
   📊 LUFS antes de normalizar: -65.66 LUFS
   ✅ Normalizado a -14.0 LUFS


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


   ✅ Archivo exportado en: /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_1_mejorado.wav

✅ Progreso: 2/8 finalizado.

🔧 Procesando: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_2.wav
   📊 LUFS antes de normalizar: -65.30 LUFS
   ✅ Normalizado a -14.0 LUFS


...

/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


   ✅ Archivo exportado en: /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_5_mejorado.wav

✅ Progreso: 6/8 finalizado.

🔧 Procesando: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_6.wav
   📊 LUFS antes de normalizar: -64.69 LUFS
   ✅ Normalizado a -14.0 LUFS


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


   ✅ Archivo exportado en: /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_6_mejorado.wav

✅ Progreso: 7/8 finalizado.

🔧 Procesando: /content/drive/MyDrive/erick_videos/0823/audios_sin_mejorar/parte_7.wav
   📊 LUFS antes de normalizar: -62.68 LUFS
   ✅ Normalizado a -14.0 LUFS
   ✅ Archivo exportado en: /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_7_mejorado.wav

✅ Progreso: 8/8 finalizado.

🏁 ¡Proceso completado!


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


In [ ]:
# Unir los fragmentos mejorados en un solo archivo
print("🔗 Uniendo fragmentos de audio mejorado...")
audio_final = AudioSegment.empty()
for fragmento in audio_mejorado_fragmentos:
    audio_final += AudioSegment.from_wav(fragmento)
audio_final.export(processed_audio_path, format="wav")
print(f"✅ Audio mejorado final guardado en: {processed_audio_path}")

🔗 Uniendo fragmentos de audio mejorado...
✅ Audio mejorado final guardado en: /content/drive/MyDrive/erick_videos/0823/audio_mejorado.wav


In [8]:
#convertir videos a aac y juntarlos en uno solo
#unirlo al video

import os
import glob
import subprocess

# Asumiendo que ya tienes definidas estas variables arriba en tu código:
# video_path           -> ruta completa del MP4 original (p.ej. ".../0823.mp4")
# video_folder         -> carpeta base (p.ej. ".../0823/")
# audio_mejorado_folder -> carpeta con los .wav mejorados (p.ej. ".../0823/audios_mejorados/")
# video_output_folder  -> carpeta donde guardarás el MP4 final (p.ej. ".../0823/videos_con_audio_mejorado/")
# final_video_path     -> ruta final de salida para el video (p.ej. ".../0823/videos_con_audio_mejorado/video_con_audio_mejorado.mp4")

# 1) Definimos carpeta para archivos AAC
aac_folder = os.path.join(video_folder, "audios_mejorados_aac")
os.makedirs(aac_folder, exist_ok=True)

# 2) Definimos ruta para el archivo concatenado final (AAC)
audio_mejorado_full = os.path.join(video_folder, "audio_mejorado_full.aac")

def convertir_wavs_a_aac():
    """Convierte todos los .wav en 'audio_mejorado_folder' a .aac (códec AAC) y los guarda en 'aac_folder'."""
    wav_files = sorted(glob.glob(os.path.join(audio_mejorado_folder, "*.wav")))
    if not wav_files:
        print(f"No se encontraron archivos WAV en '{audio_mejorado_folder}'.")
        return

    print(f"🎛 Convirtiendo {len(wav_files)} WAV(s) a AAC en: {aac_folder}")

    for wav_path in wav_files:
        base = os.path.basename(wav_path).replace(".wav", ".aac")
        aac_path = os.path.join(aac_folder, base)

        cmd = [
            "ffmpeg", "-y",
            "-i", wav_path,
            "-c:a", "aac",
            "-b:a", "192k",
            aac_path
        ]
        print(" ".join(cmd))
        subprocess.run(cmd, check=True)

    print("✅ Conversión de WAV a AAC completada.\n")


def concatenar_aac(archivo_salida=audio_mejorado_full):
    """
    Concatena todos los .aac en 'aac_folder' en un solo archivo .aac
    usando el concat demuxer de ffmpeg.
    """
    aac_files = sorted(glob.glob(os.path.join(aac_folder, "*.aac")))
    if not aac_files:
        print(f"No se encontraron archivos AAC en '{aac_folder}'.")
        return

    print(f"🎵 Concatenando {len(aac_files)} archivos AAC en: {archivo_salida}")

    # 1. Crear listita para concat demuxer
    list_file = "aac_concat_list.txt"
    with open(list_file, "w") as f:
        for aac in aac_files:
            # Formato: file '/ruta/completa'
            f.write(f"file '{os.path.abspath(aac)}'\n")

    # 2. Ejecutar ffmpeg: -f concat -safe 0 -i list_file -c copy archivo_salida
    cmd = [
        "ffmpeg", "-y",
        "-f", "concat",
        "-safe", "0",
        "-i", list_file,
        "-c", "copy",
        archivo_salida
    ]
    print(" ".join(cmd))
    subprocess.run(cmd, check=True)

    print(f"✅ Audio concatenado final: {archivo_salida}\n")


def unir_audio_con_video(
    audio_aac=audio_mejorado_full,
    video_in=video_path,
    video_out=final_video_path
):
    """
    Reemplaza el audio del video original con el AAC concatenado.
    - Copiamos el video (-c:v copy).
    - '-map' para elegir video de la 1ª entrada, audio de la 2ª.
    - '-shortest' por si hay pequeña diferencia de duración.
    """
    print(f"🎬 Reemplazando audio en el video:\n   Video in: {video_in}\n   Audio in: {audio_aac}\n   Video out: {video_out}")

    if not os.path.exists(audio_aac):
        print(f"❌ No existe el archivo de audio: {audio_aac}")
        return
    if not os.path.exists(video_in):
        print(f"❌ No existe el archivo de video: {video_in}")
        return

    cmd = [
        "ffmpeg", "-y",
        "-i", video_in,
        "-i", audio_aac,
        "-c:v", "copy",
        "-map", "0:v:0",
        "-map", "1:a:0",
        "-shortest",
        video_out
    ]
    print(" ".join(cmd))
    subprocess.run(cmd, check=True)

    print(f"✅ Video final con audio mejorado: {video_out}\n")


# -------------------------------------------------------------------------
# Ejecución de ejemplo:
# -------------------------------------------------------------------------
if __name__ == "__main__":
    # 1) Convierte todos los WAV mejorados a AAC
    convertir_wavs_a_aac()

    # 2) Concatena todos los AAC
    concatenar_aac(audio_mejorado_full)

    # 3) Une el audio concatenado con el video original
    unir_audio_con_video(
        audio_aac=audio_mejorado_full,
        video_in=video_path,
        video_out=final_video_path
    )

    print("🎉 ¡Proceso completado!")


🎛 Convirtiendo 8 WAV(s) a AAC en: /content/drive/MyDrive/erick_videos/0823/audios_mejorados_aac
ffmpeg -y -i /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_0_mejorado.wav -c:a aac -b:a 192k /content/drive/MyDrive/erick_videos/0823/audios_mejorados_aac/parte_0_mejorado.aac
ffmpeg -y -i /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_1_mejorado.wav -c:a aac -b:a 192k /content/drive/MyDrive/erick_videos/0823/audios_mejorados_aac/parte_1_mejorado.aac
ffmpeg -y -i /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_2_mejorado.wav -c:a aac -b:a 192k /content/drive/MyDrive/erick_videos/0823/audios_mejorados_aac/parte_2_mejorado.aac
ffmpeg -y -i /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_3_mejorado.wav -c:a aac -b:a 192k /content/drive/MyDrive/erick_videos/0823/audios_mejorados_aac/parte_3_mejorado.aac
ffmpeg -y -i /content/drive/MyDrive/erick_videos/0823/audios_mejorados/parte_4_mejorado.wav -c:a aac -b:a 192k /content/drive/My

In [ ]:
import os
from pydub import AudioSegment
import librosa
import numpy as np
import soundfile as sf
import whisper
import torch


# Definir carpeta del video y sus archivos
video_folder = f"{video_folder}"  # Ajusta el nombre si es diferente
audio_processed_folder = f"{video_folder}/audios_mejorados"
os.makedirs(audio_processed_folder, exist_ok=True)

# Carpeta para las transcripciones
transcription_folder = f"{video_folder}/transcripciones"
os.makedirs(transcription_folder, exist_ok=True)

# Obtener lista de archivos de audio mejorados
partes = sorted([os.path.join(audio_processed_folder, f) for f in os.listdir(audio_processed_folder) if f.endswith(".wav")])
print(f"✅ Se encontraron {len(partes)} fragmentos de audio mejorado.")

# Transcribir cada fragmento con Whisper
print("📝 Iniciando transcripción...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("base").to(device)

transcripcion_completa = ""
for i, parte in enumerate(partes):
    print(f"📝 Transcribiendo fragmento {i+1}/{len(partes)}...")
    result = model.transcribe(parte)

    transcripcion_path = f"{transcription_folder}/transcription_{i}.txt"
    with open(transcripcion_path, "w") as f:
        f.write(result["text"])

    transcripcion_completa += result["text"] + "\n"
    print(f"✅ Transcripción guardada: {transcription_path}")

# Guardar transcripción completa en la misma carpeta del video
transcription_final_path = f"{video_folder}/transcripcion_final.txt"
with open(transcription_final_path, "w") as f:
    f.write(transcripcion_completa)

print(f"✅ Transcripción completa guardada en {transcription_final_path}")


✅ Se encontraron 8 fragmentos de audio mejorado.
📝 Iniciando transcripción...


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


📝 Transcribiendo fragmento 1/8...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Transcripción guardada: /content/drive/MyDrive/erick_videos/0823/transcripciones/transcripcion.txt
📝 Transcribiendo fragmento 2/8...


...

✅ Transcripción guardada: /content/drive/MyDrive/erick_videos/0823/transcripciones/transcripcion.txt
📝 Transcribiendo fragmento 7/8...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Transcripción guardada: /content/drive/MyDrive/erick_videos/0823/transcripciones/transcripcion.txt
📝 Transcribiendo fragmento 8/8...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Transcripción guardada: /content/drive/MyDrive/erick_videos/0823/transcripciones/transcripcion.txt
✅ Transcripción completa guardada en /content/drive/MyDrive/erick_videos/0823/transcripcion_final.txt
